In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.stats.multitest import multipletests
from scipy.stats import norm

pd.set_option('future.no_silent_downcasting', True)

from modelling import *

## Load data

In [ ]:
# Load the data
data = {}
data[2020] = pd.read_csv('../data/processed/scalar/wave_1.csv')
data[2023] = pd.read_csv('../data/processed/scalar/wave_5.csv')

In [ ]:
# Include the don't know answer in case of perceived_flood_frequency
include_dont_know = False
vif_threshold = 5.0
significance_level = 0.05

## Define independent and dependent variables

In [ ]:
# Independent variables
y_vars = [
    'raise_ground_floor_level',
    'strengthen_foundations',
    'reinforce_walls_floor',
    'raise_electricity_meter',
    'install_anti_backflow_valves',
    'install_pump_drainage',
    'fix_water_barriers'
]

y_vars_ordered = {measure: i+1 for i, measure in enumerate(y_vars)}

# Dependent variables
X_vars = {
        'threat_appraisal': [
            'perceived_flood_frequency',
            'flood_worry_level',
            'experienced_flood'],
        'coping_appraisal': {
            'self_efficacy': [f'se{i+1}_{m}' for i, m in enumerate(y_vars_ordered)],
            'response_efficacy': [f're{i+1}_{m}' for i, m in enumerate(y_vars_ordered)],
            'perceived_costs': [f'pc{i+1}_{m}' for i, m in enumerate(y_vars_ordered)]},
        'adaptive_behavior': [f'adapt{i+1}_{m}_agg' for i, m in enumerate(y_vars_ordered)],
        'extra_vars': ['responsibility_perception']
    }


## Define mappings

In [ ]:
ordinal_mappings = {
    'perceived_flood_frequency': {
        'My house is completely safe': 1,
        'Less often than 1 in 500 years': 2,
        'Once in 500 years': 3,
        'Once in 200 years': 4,
        'Once in 100 years': 5,
        'Once in 50 years': 6,
        'Once in 10 years': 7,
        'Annually': 8,
        'More frequent than once per year': 9,
        "Don't know": np.nan,
    },
    'flood_worry_level': {
        'Not at all worried': 1,
        'A little worried': 2,
        'Somewhat worried': 3,
        'Quite worried': 4,
        'Very worried': 5,
    },
    'rely_on_fam_friends': {
        'Strongly agree': 1,
        'Somewhat agree': 2,
        'Neither agree nor disagree': 3,
        'Somewhat disagree': 4,
        'Strongly disagree': 5,
    },
    'rely_on_gov': {
        'Strongly agree': 1,
        'Somewhat agree': 2,
        'Neither agree nor disagree': 3,
        'Somewhat disagree': 4,
        'Strongly disagree': 5,
    },
    'responsibility_perception': {
        'Completely government': 1,
        'Mostly government': 2,
        'Equal responsibility': 3,
        'Mostly individual': 4,
        'Completely individual': 5,
    }
}

if include_dont_know:
    nominal_cols = ['experienced_flood', 'perceived_flood_frequency_dont_know']
else:
    nominal_cols = ['experienced_flood']

y_mappings = {'Will not implement': 0, 'Will implement': 1, 'Already implemented': 2}

## Compile variables and mappings into config

In [ ]:
config = {
    'variables': X_vars,
    'structural_measures': y_vars_ordered,
    'ordinal_mappings': ordinal_mappings,
    'include_dont_know': include_dont_know,
    'nominal_cols': nominal_cols,
    'y_mappings': y_mappings,
    'vif_threshold': vif_threshold,
    'significance_level': significance_level
}

## Modelling

In [ ]:
wave_2020_results, wave_2020_summary = run_wave_analysis(data[2020], 2020, config)
wave_2023_results, wave_2023_summary = run_wave_analysis(data[2023], 2023, config)

results_by_wave = {
    2020: wave_2020_results,
    2023: wave_2023_results,
}

## Multiple testing

In [ ]:
def collect_tests(results_by_wave):
    rows = []
    for wave, res_list in results_by_wave.items():
        for res in res_list:
            m = res['structural_measure']
            stats = res['full_stats']
            for _, r in stats.iterrows():
                rows.append({
                    'wave': wave,
                    'measure': m,
                    'variable': r['variable'],
                    'coef': r['coef'],
                    'se': r['se'],
                    'p': r['p'],
                    'OR': r['OR'],
                    'OR_lo': r['OR_lo'],
                    'OR_hi': r['OR_hi']
                })
    return pd.DataFrame(rows)

tests_df = collect_tests(results_by_wave)
tests_df.head()

In [ ]:
def add_wave_fdr(df):
    out = []
    for wave, g in df.groupby('wave', as_index=False):
        rej, q, _, _ = multipletests(
            g['p'].values, alpha=0.05, method='fdr_bh')
        tmp = g.copy()
        tmp['q_wave'] = q
        tmp['sig_wave'] = rej
        out.append(tmp)
    return pd.concat(out, ignore_index=True)

tests_df = add_wave_fdr(tests_df)
tests_df.head(5)

In [ ]:
q_lookup = tests_df.set_index(['wave', 'measure', 'variable'])[
    'q_wave'].to_dict()

alpha = 0.05
for wave, res_list in results_by_wave.items():
    for res in res_list:
        m = res['structural_measure']
        # columns: variable, coef, se, p, ci_lo, ci_hi, OR, OR_lo, OR_hi
        fs = res['full_stats'].copy()
        # add q and display name
        fs['q_wave'] = fs['variable'].map(
            lambda v: q_lookup.get((wave, m, v), np.nan))
        fs['display_var'] = fs['variable'].apply(display_name)

        # add flag: significant by p but NOT by q
        fs['sig_p_only'] = (
            (fs['p'] < alpha) & (fs['q_wave'] >= alpha) & (fs['variable'] != 'const')
        )

        res['full_stats'] = fs

        # significant rows by FDR within wave
        sig_rows = fs[(fs['variable'] != 'const') &
                      (fs['q_wave'] < alpha)].copy()

        # keep one row per display_var (there should be at most one anyway)
        sig_rows = sig_rows.drop_duplicates(subset=['display_var'])

        # store display names only
        res['significant_vars'] = sig_rows['display_var'].tolist()

        # build dicts keyed by display_var, values from the row
        res['significant_vars_odds_ratio'] = {
            r['display_var']: r['OR'] for _, r in sig_rows.iterrows()}
        res['significant_vars_pvalues'] = {
            r['display_var']: r['p'] for _, r in sig_rows.iterrows()}
        res['significant_vars_confidence_intervals'] = {r['display_var']: (
            r['OR_lo'], r['OR_hi']) for _, r in sig_rows.iterrows()}
        res['significant_vars_qvalues'] = {
            r['display_var']: r['q_wave'] for _, r in sig_rows.iterrows()}

        # store variables that are significant by p but not by q
        sig_p_only_rows = fs[fs['sig_p_only']]
        res['sig_by_p_not_q'] = sig_p_only_rows['display_var'].tolist()

        # Print variables that are significant by p but not by q
        if len(res['sig_by_p_not_q']) > 0:
            print(f"Wave: {wave}, Measure: {m}, Variables significant by p only (not q): {res['sig_by_p_not_q']}")

In [ ]:
results_by_wave

In [ ]:
formatted_or_matrix = create_odds_ratio_matrix(
    results_by_wave, protective_threshold=1.0)

In [ ]:
sorted_index = {'raise_electricity_meter': 'Raise electricity meter',
                   'install_anti_backflow_valves': 'Install anti-backflow valves',
                   'install_pump_drainage': 'Install pump/drainage',
                   'fix_water_barriers': 'Fix water barriers',
                   'strengthen_foundations': 'Strengthen foundations',
                   'reinforce_walls_floor': 'Reinforce walls/floor',
                   'raise_ground_floor_level': 'Raise ground floor level'}

column_names = {'self_efficacy': 'Self-efficacy',
                'responsibility_perception': 'Responsibility perception',
                'flood_worry_level': 'Flood worry level',
                'experienced_flood_Yes': 'Experienced flood',
                'perceived_flood_frequency': 'Perceived flood frequency'}

wave_2020 = formatted_or_matrix.loc[2020]
wave_2023 = formatted_or_matrix.loc[2023]

wave_2020 = wave_2020.reindex(sorted_index.keys())
wave_2020.index = [sorted_index[i] for i in wave_2020.index]
wave_2020.columns = [column_names[i] for i in wave_2020.columns]
wave_2020.drop(index=['Raise ground floor level'], inplace=True)

wave_2023 = wave_2023.reindex(sorted_index.keys())
wave_2023.index = [sorted_index[i] for i in wave_2023.index]
wave_2023.columns = [column_names[i] for i in wave_2023.columns]
wave_2023.drop(index=['Raise ground floor level'], inplace=True)

In [ ]:
wave_2020

In [ ]:
wave_2023